In [1]:
# Finetuning.py has HuggingFace's trainer class
# Idea is to not use that here to better understand what's happening

In [2]:
import numpy as np
import evaluate
import wandb
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

wandb.init(project='training-transformers')


/Users/sgondala/opt/anaconda3/envs/training-transformers/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sgondala. Use `wandb login --relogin` to force relogin


In [3]:
dataset = load_dataset("yelp_review_full")

Found cached dataset yelp_review_full (/Users/sgondala/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)
100%|██████████| 2/2 [00:00<00:00, 14.70it/s]


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

In [7]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /Users/sgondala/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-bc89f997ef5f1a04.arrow


In [8]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [9]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [10]:
tokenized_datasets['train'][0]

{'labels': 4,
 'input_ids': [101,
  2852,
  1012,
  18522,
  4107,
  2673,
  1045,
  2298,
  2005,
  1999,
  1037,
  2236,
  18742,
  1012,
  2002,
  1005,
  1055,
  3835,
  1998,
  3733,
  2000,
  2831,
  2000,
  2302,
  2108,
  9161,
  6026,
  1025,
  2002,
  1005,
  1055,
  2467,
  2006,
  2051,
  1999,
  3773,
  2010,
  5022,
  1025,
  2002,
  1005,
  1055,
  6989,
  2007,
  1037,
  2327,
  1011,
  18624,
  2902,
  1006,
  27935,
  1007,
  2029,
  2026,
  3008,
  2031,
  4541,
  2000,
  2033,
  2003,
  2200,
  2590,
  1999,
  2553,
  2242,
  6433,
  1998,
  2017,
  2342,
  5970,
  1025,
  1998,
  2017,
  2064,
  2131,
  6523,
  7941,
  2015,
  2000,
  2156,
  15744,
  2302,
  2383,
  2000,
  2156,
  2032,
  2034,
  1012,
  2428,
  1010,
  2054,
  2062,
  2079,
  2017,
  2342,
  1029,
  1045,
  1005,
  1049,
  3564,
  2182,
  2667,
  2000,
  2228,
  1997,
  2151,
  10821,
  1045,
  2031,
  2055,
  2032,
  1010,
  2021,
  1045,
  1005,
  1049,
  2428,
  5059,
  1037,
  8744,
  1012,


In [11]:
tokenized_datasets.set_format("torch")

In [12]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [13]:
tokenized_datasets['train'][0]

{'labels': tensor(4),
 'input_ids': tensor([  101,  2852,  1012, 18522,  4107,  2673,  1045,  2298,  2005,  1999,
          1037,  2236, 18742,  1012,  2002,  1005,  1055,  3835,  1998,  3733,
          2000,  2831,  2000,  2302,  2108,  9161,  6026,  1025,  2002,  1005,
          1055,  2467,  2006,  2051,  1999,  3773,  2010,  5022,  1025,  2002,
          1005,  1055,  6989,  2007,  1037,  2327,  1011, 18624,  2902,  1006,
         27935,  1007,  2029,  2026,  3008,  2031,  4541,  2000,  2033,  2003,
          2200,  2590,  1999,  2553,  2242,  6433,  1998,  2017,  2342,  5970,
          1025,  1998,  2017,  2064,  2131,  6523,  7941,  2015,  2000,  2156,
         15744,  2302,  2383,  2000,  2156,  2032,  2034,  1012,  2428,  1010,
          2054,  2062,  2079,  2017,  2342,  1029,  1045,  1005,  1049,  3564,
          2182,  2667,  2000,  2228,  1997,  2151, 10821,  1045,  2031,  2055,
          2032,  1010,  2021,  1045,  1005,  1049,  2428,  5059,  1037,  8744,
          1012,  

In [14]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

Loading cached shuffled indices for dataset at /Users/sgondala/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-f6a75b22a5d4e223.arrow


In [15]:
# Create a dataloader

In [16]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [17]:
len(train_dataloader)

125

In [18]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-cased", num_labels=5)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'classifier.wei

In [19]:
# Optimizer

from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=3e-4)

In [28]:
# Learning rate scheduler 

from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [21]:
import torch

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [22]:
model.device

device(type='mps', index=0)

In [23]:
# Now that we have a dataloader, model, optimizer, scheduler, we can train

In [24]:
# Logistics 

from tqdm.auto import tqdm
progress_bar = tqdm(range(num_training_steps))

  0%|          | 0/375 [00:00<?, ?it/s]

In [25]:
# Putting in train mode
model.train()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [26]:
metric = evaluate.load("accuracy")

In [27]:
def evaluate_model_on_dataset_and_metric(model, eval_dataloader, metric, step_number):
    # First put in eval mode
    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])

    computed_metric = metric.compute()
    # Prints as dict - {'accuracy': 0.0}
    print(computed_metric)
    wandb.log(computed_metric, step=step_number)

In [29]:
iteration = 0

for epoch in range(num_epochs):
    for batch in train_dataloader:
        # Print all key names and sizes in batch
        # print([(k, v.size()) for k, v in batch.items()])
        
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        wandb.log({"loss": loss.item()}, step=iteration)
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        
        if iteration % 25 == 0:
            # Every 25 iterations, evaluate the model
            evaluate_model_on_dataset_and_metric(model, eval_dataloader, metric, iteration)
            model.train()
        
        iteration += 1
        progress_bar.update(1)
    
    

  0%|          | 1/375 [01:50<11:25:48, 110.02s/it]

{'accuracy': 0.22}


  7%|▋         | 26/375 [05:24<4:02:58, 41.77s/it] 

{'accuracy': 0.196}


 13%|█▎        | 50/375 [06:50<19:25,  3.59s/it]  

KeyboardInterrupt: 